# Quickstart: Molmo 7B with vLLM on GCP

This notebook walks you through deploying a vLLM server with the **Molmo-7B-D-0924** model on Google Cloud Platform.

## Prerequisites

- **Google Cloud Project** with billing enabled
- **gcloud CLI** authenticated (`gcloud auth application-default login`)
- **Terraform** >= 1.5.0 installed
- **kanoa** library installed (`pip install kanoa`)

## 1. Configure Your Project

Set your GCP project ID below:

In [ ]:
import os

# Configure your GCP project
PROJECT_ID = "your-gcp-project-id"  # <-- CHANGE THIS

# Optional: Override region (default: us-central1)
REGION = "us-central1"

os.environ["TF_VAR_project_id"] = PROJECT_ID
os.environ["TF_VAR_region"] = REGION

print(f"Project: {PROJECT_ID}")
print(f"Region:  {REGION}")

## 2. Deploy Infrastructure

Initialize Terraform and deploy the Molmo preset:

In [ ]:
%%bash
cd ../infrastructure/gcp

# Initialize Terraform (only needed once)
terraform init

In [ ]:
%%bash
cd ../infrastructure/gcp

# Deploy using Molmo preset
terraform apply -var-file=presets/molmo-7b.tfvars -auto-approve

## 3. Get API Endpoint

Retrieve the vLLM server URL:

In [ ]:
import json
import subprocess

result = subprocess.run(
    ["terraform", "output", "-json"],
    cwd="../infrastructure/gcp",
    capture_output=True,
    text=True,
)
outputs = json.loads(result.stdout)

API_ENDPOINT = outputs["api_endpoint"]["value"]
print(f"API Endpoint: {API_ENDPOINT}")

## 4. Wait for Server Ready

The vLLM server takes ~3-5 minutes to download and load the model:

In [ ]:
import time

import requests


def wait_for_server(endpoint, timeout=600):
    health_url = f"{endpoint}/health"
    start = time.time()
    print(f"Waiting for server at {health_url}...")
    while time.time() - start < timeout:
        try:
            resp = requests.get(health_url, timeout=5)
            if resp.status_code == 200:
                print(f"Server ready! ({int(time.time() - start)}s)")
                return True
        except requests.RequestException:
            pass
        print(".", end="", flush=True)
        time.sleep(10)
    print(f"Timeout after {timeout}s")
    return False


wait_for_server(API_ENDPOINT)

## 5. Run Inference with kanoa

In [ ]:
from kanoa.backends.vllm import VLLMBackend

backend = VLLMBackend(api_base=API_ENDPOINT, model="allenai/Molmo-7B-D-0924")
print(f"Connected to: {backend.model}")

In [ ]:
# Simple text query
response = backend.generate(
    prompt="What is machine learning? Explain in 2 sentences.", max_tokens=100
)
print(response)

In [ ]:
# Image analysis (Molmo is multimodal!)
response = backend.generate(
    prompt="Describe this image in detail.",
    images=[
        "https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/1200px-Cat03.jpg"
    ],
    max_tokens=200,
)
print(response)

## 6. Cost Tracking

L4 GPU: ~$0.70/hour. Server has 30-min idle timeout.

## 7. Cleanup

**Important**: Destroy when done!

In [ ]:
%%bash
cd ../infrastructure/gcp
terraform destroy -var-file=presets/molmo-7b.tfvars -auto-approve